In [1]:
import pandas as pd
import seaborn as sns
import math
import numpy as np
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


In [2]:
#imputo la moda

def imputo_moda(df):
    mode_imputer = SimpleImputer(strategy='most_frequent')
    df_2 = df.copy()
    df_2.iloc[:,:] = mode_imputer.fit_transform(df_2)
    return(df_2)

#One Hot Encoding

def OHE(df,lista_colonne):

    for nome_colonna in lista_colonne:
        df_onehot_encoded = pd.get_dummies(df[nome_colonna])
        df = pd.concat([df, df_onehot_encoded], axis=1)

    df.drop(labels = lista_colonne, inplace=True,axis=1)
    return(df)
    
#trasformo true/falso in 0/1

def TrueFalsetoInt(df,lista_colonne):

    for nome_colonna in lista_colonne:
        df[nome_colonna] = df[nome_colonna]*1
    return(df)

In [5]:
def categorical_variable(df,feature_name,type_eng):

    if type_eng=="One-Hot-Encoding":
        df_onehot_encoded = pd.get_dummies(df[feature_name])

        df_new = pd.concat([df, df_onehot_encoded], axis=1)
        df_new.drop(labels = [feature_name], inplace=True,axis=1)
        return(df_new)

    if type_eng=="One-Hot-Encoding-one-less":

        df_onehot_encoded = pd.get_dummies(df[feature_name]).iloc[: , 1:] #ignora la prima colonna creata

        df_new = pd.concat([df, df_onehot_encoded], axis=1)
        df_new.drop(labels = [feature_name], inplace=True,axis=1)
        return(df_new)
    
    if type_eng=="Label-encoding":

        labelencoder = LabelEncoder()

        df_new=df.copy()
        df_new[feature_name] = labelencoder.fit_transform(df_new[feature_name])
        return(df_new)
    
    if type_eng=="Categorical":

        return(df)


In [79]:
def numerical_variable(df, feature_name, type_eng, type_sub_eng, category_boundaries):

    if type_eng=="Numerical":

        return(df)
    
    if type_eng=="Standardization":
        scaler = StandardScaler()
        df_new=df.copy()
        df_new[feature_name] = scaler.fit_transform(df_new[[feature_name]])
        return(df_new)

    if type_eng=="Min-max":
        scaler = MinMaxScaler()
        df_new=df.copy()
        df_new[feature_name] = scaler.fit_transform(df_new[[feature_name]])
        return(df_new)    
    
    if type_eng=="Category":
        df_new = df.copy()
        labels = [*range(len(category_boundaries)-1)]
        labels = [feature_name+"_"+str(s)  for s in labels]
        df_new[feature_name] = pd.cut(df_new[feature_name],category_boundaries,labels=labels,include_lowest=True)

        return(categorical_variable(df_new,feature_name,type_sub_eng))   
            



In [45]:
#Test su titanic
#import

df_train = pd.read_csv("train.csv")

#Splitto la cabina

df_cabin = df_train['Cabin'].str.split("/",expand=True)
df_cabin.columns = ['deck', 'num', 'side']

df_train = pd.concat([df_train, df_cabin], axis=1)
df_train.drop(labels = 'Cabin', inplace=True,axis=1)


In [46]:
#Definizione Variabili

variabili_categoriche = ["HomePlanet","CryoSleep","Destination","VIP","side","deck"]
variabili_numeriche = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
variabili_inutili = ['PassengerId','Name','num']

#Elimino le variabili inutili

df_train.drop(labels = ['PassengerId','Name','num'], inplace=True,axis=1)

In [61]:
#Split Train/Test

from sklearn.model_selection import train_test_split

X = df_train.drop(labels = 'Transported', axis=1)
y = (1*df_train["Transported"]).replace(0,-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#Imputo tutto con la moda -> va fatto separato tra train e test
X_train, X_test = imputo_moda(X_train),imputo_moda(X_test)

In [122]:


#Eseguo tutte le combinazioni di lavorazioni che ha senso fare

lavorazioni_categoriche = ["One-Hot-Encoding","One-Hot-Encoding-one-less","Label-encoding"]
lavorazioni_numeriche = [#"Numerical",
                            "Standardization","Min-max",
                        #["Category","One-Hot-Encoding"],
                        #["Category","One-Hot-Encoding-one-less"],
                        #["Category","Label-encoding"],
                        ]


In [123]:
#Lista Lavorazioni

from itertools import product

A=[]
for i in range(len(variabili_categoriche)-1):
    if i==0:
        A=list(product(lavorazioni_categoriche,lavorazioni_categoriche))
    else:
        A=list(product(A,lavorazioni_categoriche))
        A=[(*x,z) for x, z in A]

for i in range(len(variabili_numeriche)):
    print(len(A),i)    
    A=list(product(A,lavorazioni_numeriche))
    A=[(*x,z) for x, z in A]

df_lavorazioni = pd.DataFrame(A,columns=variabili_categoriche+variabili_numeriche)


729 0
1458 1
2916 2
5832 3
11664 4
23328 5


In [124]:
df_lavorazioni

,HomePlanet,CryoSleep,Destination,VIP,side,deck,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Standardization,Standardization
1,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Standardization,Min-max
2,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Min-max,Standardization
3,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Min-max,Min-max
4,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Min-max,Standardization,Standardization
...,...,...,...,...,...,...,...,...,...,...,...,...
46651,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Standardization,Min-max,Min-max
46652,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Min-max,Standardization,Standardization
46653,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Min-max,Standardization,Min-max
46654,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Min-max,Min-max,Standardization


In [125]:
#Adaline


In [126]:
def net_input(w_, X):
    """Calculate net input"""
    return np.dot(X, w_[1:]) + w_[0]

def activation(self, X):
    """Compute linear activation"""
    return X

def predict(self, X):
    """Return class label after unit step"""
    return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

In [136]:
#creo i vari dataframe e testo il modello

n_iter=2000
eta= 0.0000001
random_state=1



In [144]:
mlflow.end_run()

In [145]:
from sklearn.model_selection import train_test_split
import mlflow

mlflow.set_tracking_uri("http://35.203.154.207:5000")
mlflow.set_experiment("Adaline-2")



for row in range(43910,len(df_lavorazioni)):
    
    X_train_row = X_train.copy()
    X_test_row  = X_test.copy()
    mlflow.start_run(experiment_id= "4")
    mlflow.log_param("eta", eta)
    mlflow.log_param("n_iter", n_iter)
    mlflow.log_param("combinazione", row)

    for feature in df_lavorazioni.columns:

        lavorazione = df_lavorazioni.loc[row,feature]
        
        if feature in variabili_categoriche:
            X_train_row = categorical_variable(X_train_row,feature,lavorazione)
            X_test_row = categorical_variable(X_test_row,feature,lavorazione)
        else:
            X_train_row = numerical_variable(X_train_row,feature,lavorazione,1,1)
            X_test_row = numerical_variable(X_test_row,feature,lavorazione,1,1)              


    for missing_column in list(set(X_train_row.columns) - set(X_test_row.columns)):
        X_test_row[missing_column] = 0

    for missing_column in list(set(X_test_row.columns) - set(X_train_row.columns)):
        X_train_row[missing_column] = 0

    rgen = np.random.RandomState(random_state)
    w_ = rgen.normal(loc=0.0, scale=0.01, size= 1 + X_train_row.shape[1])

    for i in range(n_iter):
        output = net_input(w_ ,X_train_row)
        errors = (y_train - output)
        w_[1:] += eta * X_train_row.T.dot(errors)
        w_[0] += eta * errors.sum()


    score = sum((np.where(net_input(w_,X_test_row) >= 0.0, 1, -1)==y_test)*1)/len(np.where(net_input(w_,X_test_row) >= 0.0, 1, -1)-y_test)
    mlflow.log_metric("score",score)
    mlflow.end_run()
    print(row,score)

    


        
    

43910 0.7519171779141104
43911 0.7162576687116564
43912 0.7718558282208589
43913 0.74079754601227
43914 0.7480828220858896
43915 0.7166411042944786
43916 0.7684049079754601
43917 0.7392638036809815
43918 0.7446319018404908
43919 0.7135736196319018
43920 0.7327453987730062
43921 0.7074386503067485
43922 0.7082055214723927
43923 0.6859662576687117
43924 0.7315950920245399
43925 0.7051380368098159
43926 0.7055214723926381
43927 0.6821319018404908
43928 0.7216257668711656
43929 0.7016871165644172
43930 0.700920245398773
43931 0.6836656441717791
43932 0.718558282208589
43933 0.6978527607361963
43934 0.7028374233128835
43935 0.6817484662576687
43936 0.7714723926380368
43937 0.7434815950920245
43938 0.75
43939 0.7047546012269938
43940 0.7668711656441718
43941 0.7392638036809815
43942 0.7450153374233128
43943 0.7047546012269938
43944 0.7657208588957055
43945 0.7396472392638037
43946 0.7480828220858896
43947 0.7085889570552147
43948 0.7603527607361963
43949 0.7369631901840491
43950 0.7423312883

In [147]:
df_lavorazioni.loc[]

,HomePlanet,CryoSleep,Destination,VIP,side,deck,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Standardization,Standardization
1,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Standardization,Min-max
2,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Min-max,Standardization
3,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Standardization,Min-max,Min-max
4,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,One-Hot-Encoding,Standardization,Standardization,Standardization,Min-max,Standardization,Standardization
...,...,...,...,...,...,...,...,...,...,...,...,...
46651,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Standardization,Min-max,Min-max
46652,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Min-max,Standardization,Standardization
46653,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Min-max,Standardization,Min-max
46654,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Label-encoding,Min-max,Min-max,Min-max,Min-max,Min-max,Standardization


In [149]:
df_lavorazioni.loc[13728]

HomePlanet               One-Hot-Encoding
CryoSleep                  Label-encoding
Destination     One-Hot-Encoding-one-less
VIP                        Label-encoding
side                       Label-encoding
deck            One-Hot-Encoding-one-less
Age                               Min-max
RoomService               Standardization
FoodCourt                 Standardization
ShoppingMall              Standardization
Spa                       Standardization
VRDeck                    Standardization
Name: 13728, dtype: object

In [150]:
df_lavorazioni.loc[6976]

HomePlanet               One-Hot-Encoding
CryoSleep       One-Hot-Encoding-one-less
Destination     One-Hot-Encoding-one-less
VIP                      One-Hot-Encoding
side                     One-Hot-Encoding
deck            One-Hot-Encoding-one-less
Age                       Standardization
RoomService               Standardization
FoodCourt                 Standardization
ShoppingMall              Standardization
Spa                       Standardization
VRDeck                    Standardization
Name: 6976, dtype: object

In [ ]:


# one-hot-encoding di HomePlanet // Destination
#Destination

df_train_2 = OHE(df_train_2,['HomePlanet','Destination','side','deck'])
df_test_2 = OHE(df_test_2,['HomePlanet','Destination','side','deck'])


#trasformo True/False in 1/0

df_train_2 = TrueFalsetoInt(df_train_2,['CryoSleep','VIP','Transported'])
df_test_2 = TrueFalsetoInt(df_test_2,['CryoSleep','VIP'])


#Tolgo le colonne che non mi servono o non so come usare ancora



In [ ]:
df_train_2.to_excel('train_1.xlsx')
df_test_2.to_excel('test_1.xlsx')

In [ ]:

print(i)

5
